
# Project: Data Wrangling ( WeRateDogs)

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#gathering">Gathering Data</a></li>
<li><a href="#assess">Assissing Data</a></li>
<li><a href="#observ">Observations</a></li>
<li><a href="#conclusion">Conclusions</a></li>
</ul>

<a id='intro'></a>
The dataset the we will analyse is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. <br/>
In this project we will analyze and visualize date and extrat insights about this data, but before that we should check data for quality and tideness to be able to do so.

In [1]:

# Import the libraries needed for this project
import pandas as pd
import requests as request
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns;
import numpy as np;


<a id='gathering'></a>
## Gather

In [2]:
# read twitter-archive file
df_archive = pd.read_csv('data/twitter-archive-enhanced.csv')

In [7]:
#download image predictions file programatically
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = request.get(url)
response.content
with open(os.path.join('data/' + url.split('/')[-1]), 'wb') as file:
    file.write(response.content)

In [8]:
# read data/image-predictions.tsv after downloading
df_image = pd.read_csv('data/image-predictions.tsv', sep='\t')

In [9]:
# unfortuntly I have tried to register for twitter developer account but I have got refused message 
# ( Your Twitter developer account application was not approved.)
# so I will read data from tweet_json directly without twitter API

with open('data/tweet-json.txt') as file:
    df_twitter = pd.read_json(file, lines= True, encoding = 'utf-8')

FileNotFoundError: [Errno 2] No such file or directory: 'data/tweet-json.txt'

In [14]:
os.listdir('data/')

['image-predictions.tsv', 'tweet-json copy', 'twitter-archive-enhanced.csv']

<a id='assess'></a>
## Assess

In [ ]:
# assess twitter-archiv, select different samples to spot issues 
df_archive.sample(5)

In [ ]:
# check source columns
df_archive.source.unique()

In [ ]:
#check names
df_archive.name.unique()

In [ ]:
df_archive.info()

In [ ]:
df_archive['rating_numerator'].unique()

In [ ]:
df_archive['rating_denominator'].unique() 

In [ ]:
df_archive.describe()

In [ ]:
df_archive.rating_numerator.value_counts().sort_index()

In [ ]:
#check for duplication
df_archive[df_archive.duplicated()]

In [ ]:
df_archive[df_archive.tweet_id==10]

In [ ]:
df_image[df_image.p1_dog==True].p1.value_counts()

In [ ]:
df_image.info()

In [ ]:
df_image[df_image['p1']=='orange']

In [ ]:
df_image[df_image.duplicated()]

In [ ]:
df_twitter

In [ ]:
df_twitter.info()

In [ ]:
df_twitter['user'][1]

In [ ]:
all_columns = pd.Series(list(df_archive)+ list(df_image)+ list(df_twitter))
all_columns[all_columns.duplicated()]

## Observations
### Quality

##### twitter_archive table: 

1. tweet_id should be string, not integer.
2. Sores with (.) read incorrectly, we need to read the correct value from text column.
3. rating denamator values should be 10, there is values other than 10
4. timestamp in archive is object, should convert it to datetime
5. columns (doggo,floofer,pupper,puppo) have None instead of NaN
6. souce column have full html link, we are intrested only in values (iphone, ....)
7. missing names (None), and invalid names (a, an, O, the ....)
8. some dog has 2 stages (example tweet_id = '' has floofer and doggo)
9. retweets (text starting with RT @) should be removed.

##### image-prediction table: 
9. missing records, 2075 instead of 2355.


##### twitter table: 
10. id columns should be rename to (twitter_id)

### Tideness

##### twitter_archive table: 
1. df_archive table: columns (doggo,floofer,pupper,puppo) are dogs stages, sould be in one columns (dog_stage)

##### twitter table: 
2. Columns (source , text) are also exists in df_archive table, so we can remove them, also we are only interested in only 3 columns (id, retweet_count, favorite_count), so we can remove all other columns
3. all 3 tables should be combined into one table

## Clean
### A. Fixing Quality Issues

#### Firstly, Lets take a copy from our data so we can keep the original ones


In [ ]:
df_archive_copy = df_archive.copy()
df_image_copy = df_image.copy()
df_tweeter_copy = df_twitter.copy()


## 1.
#### Define

convert tweet_id into string in df_archive

#### Code

In [ ]:
df_archive_copy['tweet_id'] = df_archive_copy['tweet_id'].astype(str)
df_image_copy['tweet_id'] = df_image_copy['tweet_id'].astype(str)
df_twitter['id'] = df_twitter['id'].astype(str)

#### Test

In [ ]:
df_archive_copy.info()

## 2.
#### Define
Sores with (.) read incorrectly, we need to read the correct value from text column.

In [ ]:
 df_archive_copy[df_archive_copy.text.str.contains(r"(\d+\.\d+\/\d+)")][['text', 'rating_numerator']]

#### Code

In [ ]:

# extact number before point and drop the numbers after points
extracted_score_df = \
    df_archive_copy[df_archive_copy.text.str.contains(r"(\d+\.\d+\/\d+)")].text.str.extract(r"(\d+)")

In [ ]:
df_archive_copy.loc[extracted_score_df.index, 'rating_numerator'] = extracted_score_df.values

In [ ]:
df_archive_copy.rating_numerator = df_archive_copy.rating_numerator.astype('int')

#### Test

In [ ]:
 df_archive_copy[df_archive_copy.text.str.contains(r"(\d+\.\d+\/\d+)")][['text', 'rating_numerator']]

## 3.
### Define
rating denamator values should be 10, there is values other than 10

In [ ]:
df_archive_copy[df_archive_copy.rating_denominator != 10].rating_denominator.head(5)

#### Code

In [ ]:
#set denominator to 10
df_archive_copy.loc[df_archive_copy.rating_denominator != 10,'rating_denominator'] = 10

#### Test

In [ ]:
df_archive_copy[df_archive_copy.rating_denominator != 10].rating_denominator.head(5)

## 4.
### Define
timestamp in archive is object, should convert it to time


In [ ]:
df_archive_copy.info()

#### Code

In [ ]:
df_archive_copy['timestamp'] = pd.to_datetime(df_archive_copy.timestamp)


#### Test

In [ ]:
df_archive_copy.info()

## 5.
### Define
columns (doggo,floofer,pupper,puppo) have None instead of NaN


In [ ]:
df_archive_copy[df_archive_copy.doggo == 'None']

#### Code

In [ ]:
#replace None with np.nan
df_archive_copy.loc[df_archive_copy.doggo == 'None', 'doggo'] = np.nan

In [ ]:
df_archive_copy.loc[df_archive_copy.floofer == 'None', 'floofer'] = np.nan

In [ ]:
df_archive_copy.loc[df_archive_copy.pupper == 'None', 'pupper'] = np.nan

In [ ]:
df_archive_copy.loc[df_archive_copy.puppo == 'None', 'puppo'] = np.nan

#### Test

In [ ]:
df_archive_copy[['doggo', 'floofer','pupper','puppo']].head(7)

## 6.
### Define
souce column have full html link, we are intrested only in values (iphone, ....) and convert it into categorical


In [ ]:
 df_archive_copy.source.unique()

#### Code

In [ ]:
def gettext(str1):
    start = str1.find(">")+1
    end = str1.find("<", start)
    return str1[start:end]

In [ ]:
#df_archive_copy = df_archive.copy()
df_archive_copy.source = df_archive_copy.source.astype('str').apply(lambda x: gettext(x)).astype('category')

#### Test

In [ ]:
df_archive_copy.source.value_counts()

## 7.
### Define
Remove retweets and replies (retweet count and favorite count are two of the notable column omissions)

#### Code

In [ ]:
# drop columns
cols = ['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id',
           'retweeted_status_user_id', 'retweeted_status_timestamp']
df_archive_copy.drop(columns = cols, axis=1, inplace=True)

In [ ]:
df_archive_copy.columns

#### Test

In [ ]:
df_archive_copy.columns

## 8.
### Define
remove rows which not related to dogs, (the text declare .*only rate dogs)

#### Code

In [ ]:
df_archive_copy = df_archive_copy.loc[~df_archive_copy.text.str.match('.*only rate dogs', na=False)]

#### Test

In [ ]:
df_archive_copy[df_archive_copy.text.str.match('.*only rate dogs', na=False)]

## 9.
### Define
removing retweets (text starting with RT @)

#### Code

In [ ]:
df_archive_copy = df_archive_copy[~df_archive_copy.text.str.match('RT @', na=False)]

#### Test

In [ ]:
df_archive_copy[df_archive_copy.text.str.match('RT @', na=False)]

### B. Fixing Tidiness Issues

## 1.
### Define

remove unnecessary columns, keep only 3 column which we intrest in our analysis

#### Code

In [ ]:
df_tweeter_copy = df_tweeter_copy[['id', 'retweet_count', 'favorite_count']]

#rename id to tweet_id

df_tweeter_copy.rename(columns = {'id':'tweet_id'}, inplace = True)


In [ ]:
# convert tweet_id into string
df_tweeter_copy.tweet_id = df_tweeter_copy.tweet_id.astype(str)

#### Test

In [ ]:
df_tweeter_copy.head(5)

## 2.
### Define
melt columns ('doggo', 'floofer', 'pupper', 'puppo') into categarical column

In [ ]:
#df_archive_copy = df_archive.copy()
df_archive_copy

#### Code

In [ ]:
df_archive_copy.reset_index(inplace=True)

In [ ]:
for i in range(len(df_archive_copy)):
    if not(df_archive_copy.loc[i, "doggo"] is np.nan): 
        df_archive_copy.loc[i, "stage"] = df_archive_copy.loc[i, "doggo"]
        #print(i, "doggo", df_archive_copy.loc[i])
    elif not(df_archive_copy.loc[i, "floofer"] is np.nan): 
        df_archive_copy.loc[i, "stage"] = df_archive_copy.loc[i, "floofer"]
        #print(i, "floofer", df_archive_copy.loc[i])
    elif not(df_archive_copy.loc[i, "pupper"] is np.nan): 
        df_archive_copy.loc[i, "stage"] = df_archive_copy.loc[i, "pupper"]
        #print(i, "pupper", df_archive_copy.loc[i])
    elif not(df_archive_copy.loc[i, "puppo"] is np.nan): 
        df_archive_copy.loc[i, "stage"] = df_archive_copy.loc[i, "puppo"]
        #print(i, "puppo", df_archive_copy.loc[i])
    else:
        df_archive_copy.loc[i, "stage"] = None

In [ ]:
# drop the extra columns  ('doggo', 'floofer', 'pupper', 'puppo')
df_archive_copy = df_archive_copy.drop(['doggo','floofer','pupper','puppo'] , axis=1)

#### Test

In [ ]:
df_archive_copy.stage.value_counts()

## 3.
### Define
Merge 3 tables into one table

#### Code

In [ ]:
df_clean = pd.merge(df_archive_copy, df_image_copy, on = 'tweet_id')

In [ ]:
df_clean = pd.merge(df_clean, df_tweeter_copy, on = 'tweet_id')

#### Test

In [ ]:
df_clean

<a id='observ'></a>
# Observations

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
data = df_archive_copy.rating_numerator.value_counts()
data = data[data.index < 20]
sns.barplot(data.index, data.values, palette='Greens_d',ax=ax)
ax.set(xlabel='Ratings', ylabel='Frequency', title='Ratings frequency')
plt.show()


In [ ]:
data = df_archive_copy.stage.value_counts()

fig1, ax1 = plt.subplots(figsize=(12, 6))
explode = (0.0, 0.0, 0, 0)  
ax1.pie(data.values, explode=explode, labels=data.index, autopct='%1.1f%%',
        shadow=True, startangle=-50)
# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.legend()
plt.tight_layout()
plt.title("Dog Stages Percentage")
plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(12, 6))
data = df_archive_copy.source.value_counts()
sns.barplot(y=data.index, x=data.values, palette='Greens_d',ax=ax)
ax.set(xlabel='Tweet Source', ylabel='Total', title='Tweet Source Totals')
plt.show()

In [ ]:
df_clean.head(4)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

df_clean['month'] = df_clean['timestamp'].apply(lambda x:x.strftime('%Y-%m'))

data =  df_clean.groupby('month')[['retweet_count', 'favorite_count']].aggregate(np.sum)
data = data.drop('2017-08')

plt.plot(data.index, data.values)

plt.grid()
plt.xticks(rotation=45)
plt.legend(['retweet_count', 'favorite_count'])
plt.xlabel("Date")
plt.ylabel("Count")
plt.show()



<a id='conclusion'></a>
# Conclusion

after analysing WeRateDog Data we conclude : <br/>
1. percentage of pupper stage is most populate with 64% of total tweets, next is doggo with 25%.
2. most people use (Twitter for iphone) for twetter.
3. looking at trend, number of favorets and retweet generally increasing.

In [ ]:
#Saving Cleaned data into twitter_archive_enhanced.csv
df_clean.to_csv("twitter_archive_master.csv")